In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Equation import Equation
from Variable import Variable
from sympy import *
from sympy.physics.units import *
from sympy import S

In [3]:
F = Variable(force, newtons)
M = Variable(mass, kilogram)
A = Variable(acceleration, meters/second**2, 'A')

In [4]:
assert F.name == 'F'
assert F.full_name == 'force'
assert F.all_names == ('F',)
assert A.name == 'A'
assert A.all_names == ('A',)

AssertionError: 

In [5]:
nlaw = Equation(F, M*A, 'This is Newtons first law of motion')

# 1 less var
Equation.show_solved_term = False
assert nlaw(F=4*newton, M=2*kilogram) == 2*meters/second**2

# show_solved_term
Equation.show_solved_term = True
assert nlaw(F=4*newton, M=2*kilogram) == Eq(A, 2*meters/second**2)
Equation.show_solved_term = False
assert nlaw(F=4*newton, M=2*kilogram, show_solved_term=True) == Eq(A, 2*meters/second**2)
Equation.show_solved_term = True
assert nlaw(F=4*newton, M=2*kilogram, show_solved_term=False) == 2*meters/second**2
Equation.show_solved_term = False

# convert_to(solve(nlaw(F=4*newton, M=7*pound), dict=True)[0][A], A.default_unit)

# same number of vars
assert nlaw(F=4*newton, M=2*kilogram, A=2*meters/second**2)
assert not nlaw(F=4*newton, M=2*kilogram, A=3*meters/second**2)

# Custom units works
assert nlaw(F=4*newton, M=2*kilogram, units=feet/second**2) == S(2500)/381 * feet/second**2

# Too few vars works
assert nlaw(F=4*newton) == Equation(4*newton, A*M)
assert nlaw(M=2*kilogram, units=gram) == Equation(F, 2000*A*gram)

In [6]:
nlaw(A=9.81*meters/second**2, M=12*kg)

117.72*newton

In [6]:
assert nlaw.variables == {A, F, M}

In [13]:
assert nlaw.rewrite(M) == [Equation(M, F/A)]

In [18]:
nlaw.help()

Eq(F, A*M)

where:
	A is acceleration in units of meter/second**2
	M is mass in units of kilogram
	F is force in units of newton

This is Newtons first law of motion


In [19]:
nlaw.full_form()

Eq(F, A*M)

In [24]:
nlaw()

Eq(F, A*M)

In [36]:
from uncertainties import ufloat as uf
from uncertainties import ufloat

In [29]:
nlaw(F=uf(7, 3)*newton)

TypeError: can't convert an affine function (<class 'uncertainties.core.Variable'>) to float; use x.nominal_value

In [33]:
uf(7, 3)

7.0+/-3.0

In [35]:
lambdify(['mean', 'std'], ufloat, modules='uncertainties')

NameError: name 'ufloat' is not defined

In [ ]:
x, x1, x3 = symbols('x x1 x3')
L=Symbol("L")
# eqn=(x*(L**2))+(x1*(L*1))+(x3*(L**0))
# s = solve(eqn,L)
display(eqn)
lambdify([x, x1, x3], eqn)(ufloat(10,0.2), ufloat(8,0.01), ufloat(25,2))

L**2*x + L*x1 + x3

TypeError: can't convert an affine function (<class 'uncertainties.core.Variable'>) to float; use x.nominal_value

In [ ]:
f, m, a = symbols('f, m, a')
nlaw2 = lambdify([f, m, a], Eq(f, m*a))
# nlaw2(4, 2, 2)
# nlaw2(m=2*kilograms, a=2*meters/second**2, f=4*newtons)
nlaw2(m=2, a=2, f=4)

TypeError: _lambdifygenerated() missing 1 required positional argument: 'f'

In [77]:
f, m, a = symbols('f, m, a')

nlaw2 = lambdify([m, a], m*a)
# nlaw2(4, 2, 2)
# nlaw2(m=2*kilograms, a=2*meters/second**2, f=4*newtons)
# convert_to(nlaw2(m=2*kilograms, a=2*meters/second**2), newtons)
nlaw2(m=uf(2, 1), a=uf(2, .5))

4.0+/-2.23606797749979

In [80]:
from sympy.abc import x
from sympy.utilities.lambdify import implemented_function
from sympy import lambdify
f = implemented_function('f', lambda x: x+1)
lam_f = lambdify(x, f(x))
lam_f(uf(3, 2))

4.0+/-2.0

In [83]:
f(uf(3, 1))

TypeError: can't convert an affine function (<class 'uncertainties.core.Variable'>) to float; use x.nominal_value

In [ ]:
uf(2, 1)

In [ ]:
nlaw2(4*newtons, 2*kilograms, )

TypeError: _lambdifygenerated() missing 1 required positional argument: 'a'

In [ ]:
from pycertainties import calculate, Val, uncertainty
# calculate(m*a, m=Val(3, 1), a=Val(4, 2))
uncertainty(m*a, 'm', 'a').subs(dict(m=2, a=2, δm=1, δa=S(1)/2))


TypeError: type 'Union' is not subscriptable

In [117]:
def uncertainty(expr: Expr):
    return sqrt(sum(
        diff(expr, sym) ** 2 * Symbol("δ" + sym.name) ** 2
        for sym in expr.atoms(Symbol)
    ))

In [116]:
uncertainty(Eq(Symbol('f'), m*a))

sqrt(δa**2*Derivative(Eq(f, a*m), a)**2 + δf**2*Derivative(Eq(f, a*m), f)**2 + δm**2*Derivative(Eq(f, a*m), m)**2)

In [128]:
expr = solve(nlaw, F, list=True)[0]
t = sqrt(sum(
    diff(expr, sym)**2 * Symbol("δ" + sym.name)**2
    for sym in expr.atoms(Symbol)
)).subs({M:2, A:2, 'δM': 1, 'δA':1})

In [130]:
uf(2, t)

2.0+/-2.8284271247461903

A*M

NameError: name 'nlaw' is not defined